## 基于全连接神经网络的数字识别

---

#### 介绍

本实验将利用之前学到的 PyTorch 的相关知识，建立一个全连接神经网络模型，用于识别手写字符。经过本实验的学习，你将明白如何利用 PyTorch 完成数据集的预处理、数据加载器的生成、优化器的定义、损失的定义、全连接神经网络的搭建、训练与测试等过程。

#### 知识点

- 数据加载器的定义
- 优化器的定义
- 损失的定义
- 全连接网络的建立
- 模型的训练与测试

---

### 手写字符识别

在实验开始之前，我们可以利用 `device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') ` 确定当前环境是否支持 GPU 运行。

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

若 type 为 `cuda` 则表示支持 GPU，否则就是只支持 CPU。由于云服务器 GPU 的成本极高，而训练能够识别 MNIST 的神经网络并不需要太多时间，因此，这里我们没有提供 GPU 的云服务。

当然，无论是 CPU 环境和 GPU 环境，下面代码都是可以正常运行的。因为，我们已经把这个环境变量封装到 device 中了。

在下面代码中，我们会对所有数据变量添加一个 `.to(device)` 操作。如果当前环境支持 GPU 运行， `.to(device)`  就可以使变量转成可放入 GPU 中的类型。若不支持，`.to(device)`  就可以使变量转成可放入 CPU 中的类型。

#### 数据的预处理

在本课程的《数据加载器》章节，我们已经详细阐述了如何制作一个 PyTorch 认可的数据加载器。如果忘记的同学，可以先返回该章节进行一个复习。

首先让我们加载 PyTorch 中的自带数据集合，该数据集合存在于 `torchvision.datasets` 中，可以直接利用 ` torchvision.datasets.MNIST` 获得(下面代码将运行 3-5 min，请耐心等待)：

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 将数据集合下载到指定目录下,这里的transform表示，数据加载时所需要做的预处理操作
# 加载训练集合
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=torchvision.transforms.ToTensor(),
                                           download=True)
# 加载测试集合
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())
train_dataset, test_dataset

接下来，让我们把数据放入数据加载器中：

In [ ]:
batch_size = 100
# 根据数据集定义数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
train_loader, test_loader

在定义完 PyTorch 能够识别的数据加载器后，我们可以加载几张图片，观察一下图片效果：

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# 加载测试集中的前 6 张图片
examples = iter(test_loader)
example_data, example_targets = examples.next()

for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
plt.show()

本实验的任务就是利用 PyTorch 建立一个神经网络模型，用以识别上面的这种手写字符图片。

因此，模型的输入节点数和输出节点数为：

In [ ]:
 # 输入节点数就为图片的大小：28×28×1
input_size = 784
#由于数字为 0-9，因此是10分类问题，因此输出节点数为 10
num_classes = 10 
input_size,num_classes

#### 模型的建立

还记得上一章节我们所说的自定义网络模型必须满足的两个条件吗？

- 类必须继承 `nn.Module`。
- 类必须实现 `__init__` 和 `forward` 函数。

接下来，让我们利用 PyTorch 建立一个简单的神经网络模型用于手写字符的识别。

In [ ]:
# 包含了一个隐含层的全联机神经网络
class NeuralNet(nn.Module):
    # 输入数据的维度，中间层的节点数，输出数据的维度
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return  out
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 建立了一个中间层为 500 的三层神经网络，且将模型转为当前环境支持的类型（CPU 或 GPU）  
model  = NeuralNet(input_size, 500, num_classes).to(device)
model

从上面的数据结构可以知道，该神经网络模型一共有三层。

- 第一层为输入层，节点数量和图像大小相同。
- 第二次为隐藏层，节点数为 500 。
- 第三层为输出层，节点大小为 10 ，节点数大小和类别相同。

#### 损失和优化器的定义

定义完模型后，接下来，我们需要定义模型训练时所需要的损失函数和优化器。

这里我们就使用之前讲过的交叉熵损失函数：

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion

由于传统梯度下降算法存在一定的缺陷，比如学习率一直不变。因此，我们利用 PyTorch 中定义的梯度下降算法的优化算法，Adam 算法，来进行模型的训练。

In [ ]:
# 此时学习率为 0.001 ，你也可以根据实际情况，自行设置
learning_rate = 0.001
# 定义 Adam 优化器用于梯度下降
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
optimizer

#### 模型的训练

模型训练的步骤和之前实验提到的步骤一致，这些步骤可以说是固定的：

- 通过模型的正向传播，输出预测结果。
- 通过预测结果和真实标签计算损失。
- 通过后向传播，获得梯度。
- 通过梯度更新模型的权重。
- 进行梯度的清空。
- 循环上面操作，直到损失较小为止。

让我们用代码完成上面的步骤：

In [ ]:
num_epochs = 2
# 数据总长度
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # 因为全连接会把一行数据当做一条数据，因此我们需要将一张图片转换到一行上
        # 原始数据集的大小: [100, 1, 28, 28]
        # 将每一张图片都转为一行向量，
        # resize 后的向量大小: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # 正向传播以及损失的求取
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播
        # 下面三句话固定：梯度清空，反向传播，权重更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print("模型训练完成")


从结果可以看出我们的模型已经训练完毕。接下来，让我们先放入几张图片观察一下，预测结果是否和真实一致：

In [ ]:
## 测试样例
examples = iter(test_loader)
example_data, example_targets = examples.next()

## 图片的展示
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
plt.show()

# 结果的预测
images = example_data
images = images.reshape(-1, 28*28).to(device)
labels = labels.to(device)

# 正向传播以及损失的求取
outputs = model(images)
#将 Tensor 类型的变量 example_targets 转为 numpy 类型的，方便展示
print("上面三张图片的真实结果：",example_targets[0:3].detach().numpy())
# 将得到预测结果
# 由于预测结果是 N×10 的矩阵，因此利用 np.argmax 函数取每行最大的那个值，最为预测值
print("上面三张图片的预测结果：",np.argmax(outputs[0:3].detach().numpy(),axis=1))


我们简单的选取了测试集合的前三张手写字符进行测试，得到的预测结果和真实结果完全一致。

注意，这里我们 model 得到的预测结果是 one-hot 编码的。换句话说，一张图片的预测结果的大小为 $1\times 10$，我们还需要利用 `np.argmax` 求取该数组中最大数的下标。 我们可以展示一下 outputs[0] 的结果：

In [ ]:
outputs[0] 

从结果可以很清楚的看出，该数组的第 7 个值最大（从第 0 个值开始数），因此 `np.argmax(outputs[0])` 返回了 7 。

#### 模型的测试

在训练完模型后，我们将导入测试数据集，对模型进行测试，对比模型的预测结果和实际结果，进而得到模型的识别准确率。

模型的测试代码很简单，其实就是将数据传入模型之中，并进行一次正向传播即可。

简单的说，就是复制上面的模型训练代码，然后更换数据集，删除后面的梯度下降相关的代码即可。

In [ ]:
# I测试数据，计算模型的识别准确率
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        # 和训练代码一致
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        # 进行模型训练
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

从结果看出，我们利用 PyTorch 建立的三层全连接网络对手写字符图片也要较高的识别准确率，这也侧面说明了神经网络的强大。

### 实验总结

通过本实验的学习，我想你已经掌握了如何利用 PyTorch 完成数据集的预处理、数据加载器的生成、优化器的定义、损失的定义、全连接神经网络的搭建与训练以及模型的测试等过程。在下一个实验中，我们会对卷积神经网络进行详细的讲解。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>